In [1]:
import os
import pathlib
from datetime import datetime
# Setup Spark
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import when, regexp_replace, lit,format_string,current_timestamp,to_date


In [2]:
trust_path = './dados/1_trusted'
refine_path = './dados/2_refined'

In [3]:

spark = SparkSession.builder.appName("AtividadeSQL").getOrCreate()
#spark.conf.set("spark.sql.parquet.enableVectorizedReader","false").appName("AtividadeSQL")

In [4]:
teste = 'dados/1_trusted/126_tables_2022-11-27_15.39.52.csv'

In [5]:
df = spark.read.load(teste, format="csv", inferSchema=True, header=True)
df.show()
df = df.toPandas()
df.to_parquet(refine_path, engine = 'auto' ,partition_cols= ['Dados'],)
#df.coalesce(1).write.partitionBy("Dados").mode('overwrite').format("parquet").save(refine_path)

+---------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+----+----+
|    Dados|           cl_2013|           cl_2014|           cl_2015|          cl_2016|           cl_2017|           cl_2018|          cl_2019|          cl_2020|          product|unit|  uf|
+---------+------------------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+-----------------+----+----+
|  JANEIRO| 4456692.990000001| 4566320.550000001|4732998.7530000005|3942869.982999999|3959166.6520000002| 4135742.426999999|4391503.430000002|      4432971.261|ÓLEO DIESEL TOTAL|  m3|Tudo|
|FEVEREIRO| 4276021.111999999| 4679585.070000002|4071620.8389999997|      4284566.795|4034946.4359999993|       4120481.712|      4375219.448|4514231.522999998|ÓLEO DIESEL TOTAL|  m3|Tudo|
|    MARÇO| 4696752.166999999| 4815102.662999999| 50138

In [5]:
list_files = list(pathlib.Path(trust_path).glob('**/*.csv'))
list_files

[WindowsPath('dados/1_trusted/126_tables_2022-11-27_15.39.52.csv'),
 WindowsPath('dados/1_trusted/164_tables_2022-11-27_15.39.52.csv'),
 WindowsPath('dados/1_trusted/203_tables_2022-11-27_15.39.52.csv'),
 WindowsPath('dados/1_trusted/239_tables_2022-11-27_15.39.52.csv'),
 WindowsPath('dados/1_trusted/345_tables_2022-11-27_15.39.52.csv')]

In [ ]:
def unpivot_table(path):
    df = spark.read.csv(path, inferSchema=True, header=True)
    #df.write.parquet(refine_path,mode = "overwrite")
    list_columns = df.columns
    list_remove = ['Dados','product','unit','uf']
    list_columns = list(set(list_columns) - set(list_remove))
    
    for col in list_columns:
        table_name = f'dt_{col}'
        spark.catalog.dropTempView(table_name)
        
        df.createTempView(table_name)
        df2 = spark.sql(f"select Dados AS month,Dados AS month_num, uf, product, unit, {col} AS volume  from {table_name} where {col} IS NOT NULL")
        year = col.split('_')[-1]
        #print(year)
        df2 = df2.withColumn('month_num', 
        when(df2.month_num.endswith('JANEIRO'),regexp_replace(df2.month_num,'JANEIRO','01')) \
        .when(df2.month_num.endswith('FEVEREIRO'),regexp_replace(df2.month_num,'FEVEREIRO','02')) \
        .when(df2.month_num.endswith('MARÇO'),regexp_replace(df2.month_num,'MARÇO','03')) \
        .when(df2.month_num.endswith('ABRIL'),regexp_replace(df2.month_num,'ABRIL','04')) \
        .when(df2.month_num.endswith('MAIO'),regexp_replace(df2.month_num,'MAIO','05')) \
        .when(df2.month_num.endswith('JUNHO'),regexp_replace(df2.month_num,'JUNHO','06')) \
        .when(df2.month_num.endswith('JULHO'),regexp_replace(df2.month_num,'JULHO','07')) \
        .when(df2.month_num.endswith('AGOSTO'),regexp_replace(df2.month_num,'AGOSTO','08')) \
        .when(df2.month_num.endswith('SETEMBRO'),regexp_replace(df2.month_num,'SETEMBRO','09')) \
        .when(df2.month_num.endswith('OUTUBRO'),regexp_replace(df2.month_num,'OUTUBRO','10')) \
        .when(df2.month_num.endswith('NOVEMBRO'),regexp_replace(df2.month_num,'NOVEMBRO','11')) \
        .when(df2.month_num.endswith('DEZEMBRO'),regexp_replace(df2.month_num,'DEZEMBRO','12')) \
        .otherwise(df2.month_num))
        df2 =df2.select('*', lit(year).alias("year"))
        df2 =df2.select('*', current_timestamp().alias("created_at"))
        df2 = df2.withColumn('year_month',format_string("%s/%s",df2.year,df2.month_num))
        df2 = df2.withColumn('year_month',to_date(df2.year_month, "yyyy/MM"))
        df2 = df2.select('year_month','uf','product','unit','volume','created_at')

        try: 
            df_export = df_export.unionAll(df2)
        
        except:
            df_export = df2

    #df2.show()
    spark.catalog.dropTempView(table_name)

    #df_export.show()



    return df_export

In [ ]:
df = unpivot_table(teste)
df.show(1000)



    

In [ ]:
def join_csv(path_folder):

    list_files = list(pathlib.Path(path_folder).glob('**/*.csv'))
    for file in list_files:
        file = str(file)
        print(str(file))
        df = unpivot_table(file)
        
        #df.show(100)

        try:
            df_export = df_export.unionAll(df)
        except:
            df_export = df

    return df_export

    

In [ ]:
df = join_csv(trust_path)
#df.collect()
df.show()

In [ ]:
#df = df.toPandas()
#df

In [ ]:
df.to_parquet(refine_path, engine = 'pyarrow' ,partition_cols= ['year_month'],)

In [15]:
refine_path = './dados/2_refined/'

In [10]:
df.write.csv(refine_path,mode = "overwrite",sep='\t')

Py4JJavaError: An error occurred while calling o97.csv.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)
	at

In [16]:
df.write.format("parquet").mode('overwrite').save(refine_path)

Py4JJavaError: An error occurred while calling o10105.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.jobAbortedError(QueryExecutionErrors.scala:496)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:251)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:186)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:113)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:111)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.executeCollect(commands.scala:125)
	at org.apache.spark.sql.execution.QueryExecution$$anonfun$eagerlyExecuteCommands$1.$anonfun$applyOrElse$1(QueryExecution.scala:97)
	at org.apache.spark.sql.execution.SQLExecution$.$anonfun$withNewExecutionId$5(SQLExecution.scala:103)

In [6]:
spark.stop()